In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pyspark.sql.functions as sqlf

# Create a Spark Context
sc = SparkContext.getOrCreate();
sqlc = SQLContext(sc)

years = range(2000, 2001)

for year in years:
    txtfile = sc.textFile('../madhu95/data/%s.csv' % year)
    # RDD: split attribute values using commas
    data = txtfile.map(lambda x: x.split(','))
    # RDD:create table
    table = data.map(lambda r: Row(station=r[0], date=r[1], ele=r[2], val=int(r[3]), m_flag=r[4], q_flag=r[5], s_flag=r[6], obs_time=r[7]))
    #  dataframe
    df = sqlc.createDataFrame(table)

    # Handle abnomalities missing data
    clean_df = df.filter(df['q_flag'] == '')

    print("\nYear %s Temerature statistics:\n" % year)
    

    # Max TMAX
    res = clean_df.filter(clean_df['ele'] == 'TMAX').groupby().max('val').first()
    print('Max TMAX(maximum temperature) value = %.2f degrees Celsius' % (res['max(val)'] / 10.0))

    #  Min TMIN
    res = clean_df.filter(clean_df['ele'] == 'TMIN').groupby().min('val').first()
    print('Min TMIN(Minimum temperature) value = %.2f degrees Celsius' % (res['min(val)'] / 10.0))

    # Five hottest weather stations
    res = clean_df.filter(clean_df['ele'] == 'TMAX').sort(sqlf.desc('val')).limit(3).collect()
    print("Top 3 hottest weather stations  in tems of temperature ")
    for i in res:
        print('Station:%s\tTemperature:%.2f degrees Celsius' % (i.station, float(i['val']) / 10.0))
    # Five coldest weather stations 
    res = clean_df.filter(clean_df['ele'] == 'TMIN').sort(sqlf.asc('val')).limit(3).collect()
    print("Top 3 coldest weather stations in terms of temperature")
    for i in res:
        print('Station:%s\tTemperature:%.2f degrees Celsius' % (i.station, float(i['val']) / 10.0))
        
        

# overall statistics
txtfile1 = sc.textFile('../madhu95/data/20??.csv')
data = txtfile1.map(lambda x: x.split(','))
table = data.map(lambda r: Row(station=r[0], date=r[1], ele=r[2], val=int(r[3]), m_flag=r[4], q_flag=r[5], s_flag=r[6], obs_time=r[7]))
df = sqlc.createDataFrame(table)
clean_df = df.filter(df['q_flag'] == '')

# hottest day and weather station
res = clean_df.filter(clean_df['ele'] == 'TMAX').sort(sqlf.desc('val')).first()
print("Hottest station: %s on %s with temperature:%.2f degrees Celsius" % (res.station, res.date, float(res['val']) / 10.0))

# coldest day and weather station
res = clean_df.filter(clean_df['ele'] == 'TMIN').sort(sqlf.asc('val')).first()
print("Coldest Station: %s on %s with temperature:%.2f degrees Celsius" % (res.station, res.date, float(res['val']) / 10.0))